#Final Project - Weather Wiz

### Overview
  Weather Wiz is a cutting-edge machine learning project designed to predict weather patterns, specifically temperature and radiation, using 25 years of historical data collected from meteorological stations across Israel by the Israel Meteorological Service (IMS). The data was sampled every 10 minutes daily and later aggregated into a daily template.


#TODOs:
- לסדר את המידע ולהוסיף את כל הפיצרים של התחנות שחסר (קורדינטות ועוד..)
- להכניס את ה EDA לקוד ולבדוק שהועא עובד
- לסדק את ה data prep אחרי התיקון של המידע
- להמשיך לסדר את ההייפר פרמטרים
- להכניס מצגת להצגה וגם קובץ הגשה
- לשמור תמונות רלוונטיות לתרגיל כדאי להעלות לגיט
- לסדר את התצוגה של הגרפים
- לסדר את הקוד מחדש לפני הנושאים הרלוונטים

# Connect to Google drive

In [48]:
# -------------------------------------------------------------------
# Connect to Google drive
# -------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Importing Needed Packages

In [49]:
# -------------------------------------------------------------------
# Importing Needed Packages
# -------------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math

# TensorFlow
import tensorflow as tf

# Scikit-learn for Machine Learning
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

#if theres a probelm with the import.
!pip install torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.1.0+cpu.html
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

# Miscellaneous
from collections import Counter

# -------------------------------------------------------------------
# Confirmation of successful imports
# -------------------------------------------------------------------
print("All imports are successful!")

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html
All imports are successful!


In [50]:
# -------------------------------------------------------------------
# Set Seeds for Reproducibility & Device Management
# -------------------------------------------------------------------

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


# --------------------------- Helper Functions ---------------------------

In [51]:
# -------------------------------------------------------------------
# Helper Functions for Evaluation & Plotting
# -------------------------------------------------------------------

def median_absolute_percentage_error(y_true, y_pred, delta=1e-3, epsilon=1e-6):
    """
    Compute a modified median absolute percentage error (MdAPE) that ignores
    samples with |y_true| less than delta.

    Parameters:
        y_true (array-like): True target values.
        y_pred (array-like): Predicted target values.
        delta (float): Threshold below which a true value is considered too close to zero.
        epsilon (float): Small constant to prevent division by zero.

    Returns:
        float: The median absolute percentage error calculated only on samples with |y_true| > delta.
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Create a mask for valid samples
    mask = np.abs(y_true) > delta
    if np.sum(mask) == 0:
        return np.nan  # or 0, depending on your preference
    relative_errors = np.abs((y_true[mask] - y_pred[mask]) / np.maximum(np.abs(y_true[mask]), epsilon)) * 100
    return np.median(relative_errors)


def evaluate_model(model_name, y_true, y_pred):
    """
    Evaluate a model's performance using various metrics.

    Parameters:
        model_name (str): Name of the model.
        y_true (array-like): True target values.
        y_pred (array-like): Predicted target values.

    Returns:
        tuple: (mae, mse, r2, mdape)
    """
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2  = r2_score(y_true, y_pred)
    mdape = median_absolute_percentage_error(y_true, y_pred)

    print(f"{model_name} Performance:")
    print(f"  MAE: {mae:.4f}")
    print(f"  MSE: {mse:.4f}")
    print(f"  R2: {r2:.4f}")
    print(f"  MdAPE: {mdape:.4f}%\n")
    return mae, mse, r2, mdape


def train_and_evaluate(model, model_name, train_X, test_X, train_y, test_y):
    """
    Train a scikit-learn model and evaluate its performance.

    This function fits the given model on the training data, makes predictions
    on the test data, evaluates the predictions, and plots actual vs. predicted values.

    Parameters:
        model: scikit-learn model instance.
        model_name (str): Name of the model (for printing and plotting).
        train_X: Training feature data.
        test_X: Testing feature data.
        train_y: Training target values.
        test_y: Testing target values.

    Returns:
        tuple: (mae, mse, r2, MdAPE, predictions)
    """
    # Train the model
    model.fit(train_X, train_y)
    predictions = model.predict(test_X)
    predictions = np.round(predictions, 2)

    # Evaluate the model
    mae, mse, r2, mdape = evaluate_model(model_name, test_y, predictions)

    # Plot Actual vs. Predicted values
    plt.figure(figsize=(8, 6))
    plt.scatter(test_y, predictions, alpha=0.6, label="Predicted vs Actual")
    plt.plot([min(test_y), max(test_y)], [min(test_y), max(test_y)], 'r--', label="Perfect Fit")
    plt.xlabel("Actual Values")
    plt.ylabel("Predicted Values")
    plt.title(f"{model_name} - Actual vs Predicted")
    plt.legend()
    plt.show()

    return mae, mse, r2, mdape, predictions


def plot_series(test_obs, test_preds, model_name, fold_idx):
    test_obs.index = pd.to_datetime(test_obs.index)
    test_preds = pd.Series(test_preds, index=test_obs.index)
    fig, ax = plt.subplots(figsize=(20, 6))
    ax.plot(test_obs, label="Observation", lw=3, linestyle="--")
    ax.plot(test_preds, label="Prediction")
    ax.set_title(f"{model_name}, Fold: {fold_idx} - Observation vs Prediction")
    ax.legend()
    ax.set_xlabel("Day")
    ax.grid(True)
    plt.xticks(rotation=90)
    plt.show()


# -------------------------------------------------------------------
# Helper Functions for Creating Sequences & DataLoaders
# -------------------------------------------------------------------
def create_time_series_sequences(features, targets, sequence_length):
    if isinstance(features, (pd.DataFrame, pd.Series)):
        features = features.values
    if isinstance(targets, pd.Series):
        targets = targets.values
    X_sequences, y_sequences = [], []
    for i in range(len(features) - sequence_length):
        X_sequences.append(features[i : i + sequence_length])
        y_sequences.append(targets[i + sequence_length])
    return np.array(X_sequences), np.array(y_sequences)


def create_data_loader(X, y, batch_size=32, shuffle=True):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)
    dataset = TensorDataset(X_tensor, y_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)


def create_transformer_input_sequences(features, targets, sequence_length):
    return create_time_series_sequences(features, targets, sequence_length)


def create_transformer_data_loader(X, y, batch_size=32, shuffle=True):
    return create_data_loader(X, y, batch_size=batch_size, shuffle=shuffle)


# --------------------------- Data Preparation & Time series split ---------------------------



In [52]:
# -------------------------------------------------------------------
# Updated Data Preparation Functions
# -------------------------------------------------------------------

def custom_time_series_split(df, n_splits, test_size):
    X = df.drop(columns=["station_id", "TD"], errors="ignore")
    y = df["TD"].copy()

    X_train_list, y_train_list = [], []
    X_test_list, y_test_list = [], []
    train_idx_list, test_idx_list = [], []

    from sklearn.model_selection import TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=n_splits, test_size=test_size)
    for fold, (train_index, test_index) in enumerate(tscv.split(X)):
        print(f"Fold {fold+1}: TRAIN indices=[{train_index[0]}..{train_index[-1]}], TEST=[{test_index[0]}..{test_index[-1]}]")
        X_train = X.iloc[train_index]
        y_train = y.iloc[train_index]
        X_test  = X.iloc[test_index]
        y_test  = y.iloc[test_index]

        # Append the indices for later use
        train_idx_list.append(train_index)
        test_idx_list.append(test_index)

        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        scaler.fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        X_train_list.append(X_train_scaled)
        y_train_list.append(y_train)
        X_test_list.append(X_test_scaled)
        y_test_list.append(y_test)

    # Return both the data splits and the indices splits
    return X_train_list, y_train_list, X_test_list, y_test_list, train_idx_list, test_idx_list


def prepare_data(df, n_splits, test_size_ratio):
    """
    Prepare the data:
      - Convert 'datetime' column to datetime objects.
      - Sort by datetime.
      - Normalize the 'RH' column.
      - Replace invalid values and fill missing data.
      - Optionally drop 'TW'.
      - Set datetime as index.
      - Compute test set size using a ratio and a heuristic.
      - Split the data using custom_time_series_split.

    Parameters:
        df : DataFrame with a 'datetime' column.
        n_splits (int): Number of splits.
        test_size_ratio (float): Desired test set ratio.

    Returns:
        X_train_list, y_train_list, X_test_list, y_test_list for each fold.
    """
    df = df.copy()
    df['datetime'] = pd.to_datetime(df['datetime'], utc=True, errors='coerce')
    df = df.sort_values('datetime')

    df["RH"] = df["RH"] / 100.0
    df.replace(-9999.0, np.nan, inplace=True)

    if 'TW' in df.columns:
        df.drop(columns='TW', inplace=True)

    for col in df.columns[df.isna().any()]:
        df[col] = df[col].fillna(df[col].mean())

    df.set_index('datetime', inplace=True)

    test_size = int(len(df) * test_size_ratio)
    max_test_size = len(df) // (n_splits * 2)
    if test_size < max_test_size:
        test_size = max_test_size

    print(f"Using test_size={test_size} ({(test_size/len(df))*100:.1f}% of dataset)")
    return custom_time_series_split(df, n_splits, test_size)

# --------------------------- Linear Models (Benchmark) ---------------------------

## Note: keep the model with the best results when the Data is completed
Keep the model that demonstrates the best balance of:

- Generalization performance (lowest MAE/MSE, highest R² on unseen data),
- Stability (consistent performance across folds), and
- Interpretability (if needed, the simpler or sparser model might be preferred).

In [53]:
# Linear Regression - No regularization
# Formula: y = Xw + b
def linear_regression_model(train_X, test_X, train_y, test_y, **kwargs):
    print("\n--- Training Linear Regression Model ---")
    model = LinearRegression(**kwargs)
    return train_and_evaluate(model, "Linear Regression", train_X, test_X, train_y, test_y)

# Ridge Regression - L2 regularization
# Formula: Minimize ||y - Xw||^2 + alpha * ||w||^2
def ridge_regression_model(train_X, test_X, train_y, test_y, **kwargs):
    print("\n--- Training Ridge Regression Model ---")
    model = Ridge(**kwargs)
    return train_and_evaluate(model, "Ridge Regression", train_X, test_X, train_y, test_y)

# Lasso Regressio - L1 regularization
# Formula: Minimize ||y - Xw||^2 + alpha * ||w||_1
def lasso_regression_model(train_X, test_X, train_y, test_y, **kwargs):
    print("\n--- Training Lasso Regression Model ---")
    model = Lasso(**kwargs)
    return train_and_evaluate(model, "Lasso Regression", train_X, test_X, train_y, test_y)

# --------------------------- Random Forest Model ---------------------------


In [54]:
def random_forest_model(train_X, test_X, train_y, test_y, **kwargs):
    print("\n--- Training Random Forest Model ---")
    model = RandomForestRegressor(**kwargs)
    return train_and_evaluate(model, "Random Forest", train_X, test_X, train_y, test_y)

# --------------------------- LSTM Model ---------------------------


In [55]:
# -------------------------------------------------------------------
# LSTM Model & Training Functions
# -------------------------------------------------------------------

class WeatherForecastLSTM(nn.Module):
    def __init__(self, input_dim, hidden_units=128, lstm_layers=2, dropout_rate=0.3):
        super(WeatherForecastLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_units,
            num_layers=lstm_layers,
            dropout=dropout_rate if lstm_layers > 1 else 0,
            batch_first=True,
            bidirectional=True
        )
        self.fc = nn.Linear(hidden_units * 2, 1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        final_output = self.fc(lstm_out[:, -1, :])
        return final_output

def train_lstm_model(model, train_loader, val_loader, loss_function, optimizer,
                     epochs=50, patience=10, save_path="best_lstm_model.pth"):
    best_val_loss = float('inf')
    patience_counter = 0
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                           patience=5, factor=0.5, verbose=True)
    model.to(device)
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            predictions = model(batch_X)
            loss = loss_function(predictions, batch_y)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_train_loss += loss.item()
        avg_train_loss = total_train_loss / len(train_loader)

        model.eval()
        total_val_loss = 0.0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                predictions = model(batch_X)
                loss = loss_function(predictions, batch_y)
                total_val_loss += loss.item()
        avg_val_loss = total_val_loss / len(val_loader)
        scheduler.step(avg_val_loss)
        print(f"LSTM Epoch {epoch+1}/{epochs} - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            torch.save(model.state_dict(), save_path)
            print("New best LSTM model saved.")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping for LSTM triggered.")
                break
    model.load_state_dict(torch.load(save_path))
    return model

def train_and_evaluate_lstm_model(train_X, test_X, train_y, test_y, sequence_length=144, **kwargs):
    batch_size = kwargs.get("batch_size", 16)
    hidden_units = kwargs.get("hidden_units", 128)
    lstm_layers = kwargs.get("lstm_layers", 2)
    dropout_rate = kwargs.get("dropout_rate", 0.3)
    learning_rate = kwargs.get("learning_rate", 0.001)
    epochs = kwargs.get("epochs", 50)

    train_X_seq, train_y_seq = create_time_series_sequences(train_X, train_y, sequence_length)
    test_X_seq, test_y_seq = create_time_series_sequences(test_X, test_y, sequence_length)

    split_index = int(len(train_X_seq) * 0.8)
    train_loader = create_data_loader(train_X_seq[:split_index], train_y_seq[:split_index], batch_size=batch_size, shuffle=True)
    val_loader   = create_data_loader(train_X_seq[split_index:], train_y_seq[split_index:], batch_size=batch_size, shuffle=False)
    test_loader  = create_data_loader(test_X_seq, test_y_seq, batch_size=batch_size, shuffle=False)

    input_dim = train_X_seq.shape[2]
    model = WeatherForecastLSTM(input_dim, hidden_units, lstm_layers, dropout_rate)

    loss_function = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

    model = train_lstm_model(model, train_loader, val_loader, loss_function, optimizer,
                             epochs=epochs, patience=10, save_path="best_lstm_model.pth")
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch_X, _ in test_loader:
            batch_X = batch_X.to(device)
            batch_preds = model(batch_X)
            predictions.extend(batch_preds.squeeze().cpu().numpy())
    return evaluate_model("LSTM", test_y[sequence_length:], predictions) + (predictions,)

# --------------------------- Transformer Model ---------------------------

In [56]:
# -------------------------------------------------------------------
# Transformer Model & Training Functions
# -------------------------------------------------------------------

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # shape (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim, d_model=128, num_heads=4, num_layers=2, dropout=0.1):
        super(TimeSeriesTransformer, self).__init__()
        self.input_layer = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=num_heads,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.dropout = nn.Dropout(dropout)
        self.output_layer = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.input_layer(x)
        x = self.pos_encoder(x)
        x = self.dropout(x)
        x = self.transformer_encoder(x)
        x = self.output_layer(x[:, -1, :])
        return x

def train_transformer(model, train_loader, val_loader, loss_function, optimizer,
                      epochs=50, patience=10, save_path="best_transformer.pth"):
    best_val_loss = float('inf')
    patience_counter = 0
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                           patience=5, factor=0.5, verbose=True)
    model.to(device)
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            predictions = model(batch_X)
            loss = loss_function(predictions, batch_y)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        avg_train_loss = total_train_loss / len(train_loader)

        model.eval()
        total_val_loss = 0.0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                predictions = model(batch_X)
                loss = loss_function(predictions, batch_y)
                total_val_loss += loss.item()
        avg_val_loss = total_val_loss / len(val_loader)
        scheduler.step(avg_val_loss)
        print(f"Transformer Epoch {epoch+1}/{epochs} - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            torch.save(model.state_dict(), save_path)
            print("New best Transformer model saved.")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping for Transformer triggered.")
                break
    model.load_state_dict(torch.load(save_path))
    return model

def train_and_evaluate_transformer_model(train_X, test_X, train_y, test_y, sequence_length=144, **kwargs):
    batch_size = kwargs.get("batch_size", 16)
    d_model = kwargs.get("d_model", 128)
    num_heads = kwargs.get("num_heads", 4)
    num_layers = kwargs.get("num_layers", 2)
    dropout = kwargs.get("dropout", 0.1)
    learning_rate = kwargs.get("learning_rate", 0.001)
    epochs = kwargs.get("epochs", 50)

    train_X_seq, train_y_seq = create_transformer_input_sequences(train_X, train_y, sequence_length)
    test_X_seq, test_y_seq = create_transformer_input_sequences(test_X, test_y, sequence_length)

    split_index = int(len(train_X_seq) * 0.8)
    train_loader = create_transformer_data_loader(train_X_seq[:split_index], train_y_seq[:split_index], batch_size=batch_size, shuffle=True)
    val_loader   = create_transformer_data_loader(train_X_seq[split_index:], train_y_seq[split_index:], batch_size=batch_size, shuffle=False)
    test_loader  = create_transformer_data_loader(test_X_seq, test_y_seq, batch_size=batch_size, shuffle=False)

    input_dim = train_X_seq.shape[2]
    model = TimeSeriesTransformer(input_dim, d_model, num_heads, num_layers, dropout)

    loss_function = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

    model = train_transformer(model, train_loader, val_loader, loss_function, optimizer,
                              epochs=epochs, patience=10, save_path="best_transformer.pth")

    model.eval()
    predictions = []
    with torch.no_grad():
        for batch_X, _ in test_loader:
            batch_X = batch_X.to(device)
            batch_preds = model(batch_X)
            predictions.extend(batch_preds.squeeze().cpu().numpy())
    return evaluate_model("Transformer", test_y[sequence_length:], predictions) + (predictions,)

# --------------------------- Graph Neural Network Model ---------------------------



In [57]:
# -------------------------------------------------------------------
# Graph Neural Network (GNN) Model & Training Functions
# -------------------------------------------------------------------
def create_graph_data_with_window(train_X, train_y, test_X, test_y, window=2):
    """
    Build graph data using a temporal window.
    Each node is connected to the next 'window' nodes.
    This is a simple modification to provide richer temporal connectivity.
    """
    import torch
    from torch_geometric.data import Data

    # Convert inputs to numpy arrays
    train_X = np.array(train_X)
    test_X  = np.array(test_X)
    train_y = np.array(train_y)
    test_y  = np.array(test_y)

    def build_edges(num_nodes, window):
        edges = []
        for i in range(num_nodes):
            for j in range(1, window+1):
                if i+j < num_nodes:
                    edges.append([i, i+j])
        return edges

    train_edge_list = build_edges(len(train_X), window)
    test_edge_list  = build_edges(len(test_X), window)

    print(f"Created {len(train_edge_list)} edges for training with window={window}.")
    print(f"Created {len(test_edge_list)} edges for testing with window={window}.")

    train_edge_index = torch.tensor(train_edge_list, dtype=torch.long).t().contiguous() if train_edge_list else torch.empty((2,0), dtype=torch.long)
    test_edge_index  = torch.tensor(test_edge_list, dtype=torch.long).t().contiguous() if test_edge_list else torch.empty((2,0), dtype=torch.long)

    train_X_tensor = torch.tensor(train_X, dtype=torch.float32)
    test_X_tensor  = torch.tensor(test_X, dtype=torch.float32)
    train_y_tensor = torch.tensor(train_y, dtype=torch.float32)
    test_y_tensor  = torch.tensor(test_y, dtype=torch.float32)

    train_data = Data(x=train_X_tensor, edge_index=train_edge_index, y=train_y_tensor)
    test_data  = Data(x=test_X_tensor, edge_index=test_edge_index, y=test_y_tensor)

    return train_data, test_data

# first ver
# class GraphNeuralNetwork(nn.Module):
#     def __init__(self, num_features, hidden_dim=64, dropout=0.5):
#         super(GraphNeuralNetwork, self).__init__()
#         self.conv1 = GCNConv(num_features, hidden_dim)
#         self.bn1 = nn.BatchNorm1d(hidden_dim)
#         self.conv2 = GCNConv(hidden_dim, hidden_dim)
#         self.bn2 = nn.BatchNorm1d(hidden_dim)
#         self.conv3 = GCNConv(hidden_dim, hidden_dim)
#         self.bn3 = nn.BatchNorm1d(hidden_dim)
#         self.fc    = nn.Linear(hidden_dim, 1)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x, edge_index):
#         x = self.conv1(x, edge_index)
#         x = self.bn1(x)
#         x = F.relu(x)
#         x = self.dropout(x)
#         x = self.conv2(x, edge_index)
#         x = self.bn2(x)
#         x = F.relu(x)
#         x = self.dropout(x)
#         x = self.conv3(x, edge_index)
#         x = self.bn3(x)
#         x = F.relu(x)
#         x = self.dropout(x)
#         x = self.fc(x)
#         return x.squeeze()

# second ver
class GraphNeuralNetwork(nn.Module):
    def __init__(self, num_features, hidden_dim=64, dropout=0.5, negative_slope=0.01):
        super(GraphNeuralNetwork, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.bn3 = nn.BatchNorm1d(hidden_dim)
        self.fc = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)
        self.negative_slope = negative_slope  # Parameter for LeakyReLU

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.leaky_relu(x, negative_slope=self.negative_slope)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.leaky_relu(x, negative_slope=self.negative_slope)
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        x = self.bn3(x)
        x = F.leaky_relu(x, negative_slope=self.negative_slope)
        x = self.dropout(x)
        x = self.fc(x)
        return x.squeeze()


def train_gnn(model, train_data, epochs=50, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.MSELoss()
    model.to(device)
    train_data = train_data.to(device)
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        predictions = model(train_data.x, train_data.edge_index)
        loss = loss_function(predictions, train_data.y)
        loss.backward()
        optimizer.step()
        print(f"GNN Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")
    return model

def evaluate_gnn_final(model, test_data, test_y):
    model.eval()
    with torch.no_grad():
        predictions = model(test_data.x.to(device), test_data.edge_index.to(device)).cpu().numpy()
    mae_val  = mean_absolute_error(test_y, predictions)
    mse_val  = mean_squared_error(test_y, predictions)
    r2_val   = r2_score(test_y, predictions)
    mape_val = median_absolute_percentage_error(test_y, predictions)
    print("GNN Performance:")
    print(f"  MAE: {mae_val:.4f}")
    print(f"  MSE: {mse_val:.4f}")
    print(f"  R2 Score: {r2_val:.4f}")
    print(f"  MdAPE: {mape_val:.4f}%\n")
    return mae_val, mse_val, r2_val, mape_val, predictions

def train_and_evaluate_gnn_model(train_X, test_X, train_y, test_y, station_ids=None, **kwargs):
    hidden_dim = kwargs.get("hidden_dim", 64)
    dropout = kwargs.get("dropout", 0.5)
    learning_rate = kwargs.get("learning_rate", 0.001)
    epochs = kwargs.get("epochs", 50)

    print("\n--- Preparing Graph Data ---")
    train_data, test_data = create_graph_data_with_window(train_X, train_y, test_X, test_y, window=5)
    print("\n--- Training GNN ---")
    model = GraphNeuralNetwork(num_features=train_X.shape[1], hidden_dim=hidden_dim, dropout=dropout)
    model = train_gnn(model, train_data, epochs=epochs, lr=learning_rate)
    print("\n--- Evaluating GNN ---")
    mae_val, mse_val, r2_val, mape_val, predictions = evaluate_gnn_final(model, test_data, test_y)
    return mae_val, mse_val, r2_val, mape_val, predictions

# --------------------------- EDA ---------------------------
# EVALUATION METRICS:
## **1. Mean Absolute Error (MAE)**
- Measures the average absolute difference between predictions and actual values.
- **Ideal:** Lower is better (0 = perfect predictions).  
---

## **2. Mean Squared Error (MSE)**
- Calculates the average squared differences, penalizing larger errors.
- **Ideal:** Lower is better (0 = perfect predictions).  
---

## **3. R² Score (Coefficient of Determination)**
- Indicates how well the model explains variance in the target variable.
- **Ideal:** Higher is better (**1** = perfect fit, **0** = baseline, negative = poor fit).  
---

## **4. Median Absolute Percentage Error (MdAPE)**
- Measures the average percentage error, providing a relative error measure.
- **Ideal:** Lower is better (0% indicates perfect predictions).  
---

# **Ideal Metric Summary**

| **Metric**   | **Goal**            | **Description**                   |
|-------------|---------------------|-----------------------------------|
| **MAE**     | Lower is better     | Average absolute error           |
| **MSE**     | Lower is better     | Penalizes large errors more      |
| **R² Score** | Higher is better   | Variance explained by the model  |
| **MdAPE**    | Lower is better     | Median Absolute Percentage Error |


In [58]:
# Tomers code

#**Recommended `sequence_length` Based on Your Goal**
The dataset consists of **10-minute interval weather data** spanning **25 years**. The `sequence_length` determines how many past time steps the model should use to make predictions.

| **Goal** | **Recommended `sequence_length`** | **Explanation** |
|----------|---------------------------------|---------------------------------------------------------------|
| Predict next 1-6 hours | **6 - 36** | Uses recent short-term fluctuations (e.g., sudden weather changes). |
| Predict next 12 hours | **72** | Captures half-day variations (e.g., temperature and humidity shifts). |
| Predict tomorrow’s weather | **144** (1 day) | Accounts for daily cycles (e.g., daytime vs. nighttime temperature). |
| Predict the next 3 days | **432** (3 days) | Captures short-term trends, good for local weather forecasts. |
| Capture weekly patterns | **1008** (7 days) | Considers weekly trends (e.g., regular temperature changes, weekly patterns). |
| Predict the next 10 days | **1440** (10 days) | Captures longer short-term trends, useful for medium-range forecasting. |
| Predict the next 14 days | **2016** (14 days) | Captures bi-weekly trends, useful for extended forecasts. |
| Predict monthly variations | **4320** (30 days) | Considers longer-term weather fluctuations, such as seasonal shifts. |
| **Capture seasonal changes** | **12960** (3 months) | Tracks full-season trends like winter, summer, monsoon. |
| **Capture yearly variations** | **52560** (1 year) | Useful for long-term climate trend analysis. |

In [59]:
def get_sequence_length(forecast_horizon):
    """
    Determines the appropriate sequence_length based on the forecast horizon.

    Args:
        forecast_horizon (str): Forecasting time frame ("hourly", "daily", "weekly", "seasonal", "yearly").

    Returns:
        int: Number of past time steps to use as input.
    """
    sequence_lengths = {
        "hourly": 6,        # 1-hour (6 x 10-minute intervals)
        "short": 144,       # 1-day (144 x 10-minute intervals)
        "3-day": 432,       # 3 days (3 × 144)
        "weekly": 1008,     # 7 days (7 × 144)
        "10-day": 1440,     # 10 days (10 × 144)
        "bi-weekly": 2016,  # 14 days (14 × 144)
        "monthly": 4320,    # 30 days (30 × 144)
        "seasonal": 12960,  # 90 days (3 months × 30 days × 144)
        "yearly": 52560     # 1 year (365 × 144)
    }

    if forecast_horizon not in sequence_lengths:
        raise ValueError(f"Invalid forecast_horizon: {forecast_horizon}. Choose from {list(sequence_lengths.keys())}.")

    return sequence_lengths[forecast_horizon]


# --------------------------- Main ---------------------------

this part in the code -Main function- is responseble to load the data, train all models, and compare their performances.
    
This part includes the activation and evaluation of all the models defined above:

  * Linear Regression, Ridge Regression, and Lasso Regression (Linear Models)
  * K-Nearest Neighbors (KNN)
  * Decision Tree and Random Forest (Tree-Based Models)
  * Deep learning - Neural Network  
    - Multi-Layer Perceptron (MLP) - using ReLU-activation and backpropagation
    - Long Short-Term Memory (LSTM)
    - and more

    
After training, the models' performances are evaluated and compared using MAE, MSE, and R2 Score.
The results are visualized for a clearer comparison.

In [60]:
# keep for later - df = pd.read_csv('/content/drive/MyDrive/Course70938/earthML/finalProject/Data/IMS/allStations_10min.csv', on_bad_lines="skip")  # Noam's dataset path

In [61]:
# -------------------------------------------------------------------
# Main function
# -------------------------------------------------------------------

def main(n_splits=3, test_size_ratio=0.1, forecast_horizon="short"):
    """
    Main function to:
    - Load and preprocess data
    - Perform time-series split
    - Train multiple models (Linear, LSTM, Transformer, GNN, etc.)
    - Evaluate performance
    - Plot results

    Args:
        n_splits (int): Number of splits for TimeSeries cross-validation.
        test_size_ratio (float): Ratio of dataset to use for testing (e.g., 0.1 = 10%).
        forecast_horizon (str): "short" for short-term forecasting (1 day), "long" for long-term forecasting (14 days).
    """
    # Load Dataset
    df = pd.read_csv('/content/drive/MyDrive/earth/earthML/finalProject/Data/IMS/allStations_10min_TEMP.csv', on_bad_lines="skip")# Tomer path
    # df = pd.read_csv('/content/drive/MyDrive/Course70938/earthML/finalProject/Data/IMS/allStations_10min_TEMP.csv', on_bad_lines="skip")  # test path
    print(f"Initial shape of data: {df.shape}")

### delete later:
    # print(df.columns)
    # print(df["TD"].describe())
    # plt.hist(df["TD"], bins=50)
    # plt.xlabel("TD")
    # plt.ylabel("Frequency")
    # plt.title("Distribution of TD")
    # plt.show()
### delete later:

    # Determine sequence_length based on forecast_horizon
    sequence_length = get_sequence_length(forecast_horizon)

    # Automatically Calculate Test Size
    test_size = int(len(df) * test_size_ratio)
    if test_size < sequence_length:
        print(f"Warning: test_size={test_size} is too small for sequence_length={sequence_length}. Adjusting...")
        test_size = sequence_length + 10000  # Adjust to a safe margin
    else:
        print(f"Using test_size={test_size} ({test_size_ratio * 100}% of dataset) with sequence_length={sequence_length} for forecast_horizon='{forecast_horizon}'.")

    # Prepare Data
    X_train_list, y_train_list, X_test_list, y_test_list, train_idx_list, test_idx_list = prepare_data(df, n_splits, test_size_ratio)
    print(f"Prepared {len(X_train_list)} folds of data.")

    # Dictionary containing hyperparameters for each model
    model_hyperparams = {
        "Linear Regression": {},
        "Ridge Regression": {"alpha": 1.0},
        "Lasso Regression": {"alpha": 0.1},
        "Random Forest": {"n_estimators": 200, "max_depth": 50, "min_samples_split": 4, "random_state": 0},
        "LSTM": {
            "hidden_units": 128,
            "lstm_layers": 3,
            "dropout_rate": 0.3,
            "batch_size": 256,
            "learning_rate": 0.001,
            "epochs": 50
        },
        "Transformer": {
            "d_model": 128,
            "num_heads": 4,
            "num_layers": 2,
            "dropout": 0.2,
            "batch_size": 256,
            "learning_rate": 0.001,
            "epochs": 50
        },
        "Graph Neural Network": {
            "hidden_dim": 256,
            "batch_size": 32,
            "learning_rate": 0.005,
            "epochs": 100
        }
    }
        # 18.1
        #     "Graph Neural Network": {
        #     "hidden_dim": 256,
        #     "batch_size": 32,
        #     "learning_rate": 0.01,
        #     "epochs": 100
        # }

    # Initialize results dictionary
    results = {
        'Model': [],
        'Fold': [],
        'MAE': [],
        'MSE': [],
        'R2 Score': [],
        'MdAPE': []
    }

    # Define models
    model_funcs = {
        # 'Linear Regression': linear_regression_model,
        # 'Ridge Regression': ridge_regression_model,
        # 'Lasso Regression': lasso_regression_model,
        # 'Random Forest': random_forest_model,
        # 'LSTM': train_and_evaluate_lstm_model,
        # 'Transformer': train_and_evaluate_transformer_model,
        'Graph Neural Network': train_and_evaluate_gnn_model
    }

    # Train and evaluate each model for each fold
    for fold_idx in range(len(X_train_list)):
        print(f"\n================= Fold {fold_idx + 1} =================")
        train_X, test_X = X_train_list[fold_idx], X_test_list[fold_idx]
        train_y, test_y = y_train_list[fold_idx], y_test_list[fold_idx]

        for model_name, model_func in model_funcs.items():
            print(f"\n--- {model_name} ---")
            try:
                hyperparams = model_hyperparams.get(model_name, {})

                if model_name in {'LSTM', 'Transformer'}:
                    mae, mse, r2, mdape_val, prediction = model_func(train_X, test_X, train_y, test_y,
                                                                      sequence_length=sequence_length, **hyperparams)
                    start_index = test_y.index[sequence_length:] if hasattr(test_y, 'index') else range(sequence_length, len(test_y))
                    test_preds = pd.Series(prediction, index=test_y.index[start_index[0]:] if hasattr(test_y, 'index') else None)
                    test_obs_sliced = test_y.loc[start_index] if hasattr(test_y, 'index') else test_y
                    plot_series(test_obs_sliced, test_preds, model_name, fold_idx + 1)
                elif model_name == "Graph Neural Network":
                    # Option A: Use station-based grouping (if you want to use station IDs or Stations Coordinations)
                    # train_station_ids = df["station_id"].iloc[train_idx_list[fold_idx]].values
                    # test_station_ids  = df["station_id"].iloc[test_idx_list[fold_idx]].values
                    # unique_edge = (train_station_ids, test_station_ids)

                    # Option B: Use sequential edges with a temporal window (recommended to test improved connectivity)
                    unique_edge = None  # Use sequential edges

                    mae, mse, r2, mdape_val, predictions = model_func(train_X, test_X, train_y, test_y, unique_edge, **hyperparams)
                    test_preds = pd.Series(predictions, index=test_y.index)
                    plot_series(test_y, test_preds, model_name, fold_idx+1)
                else:
                    mae, mse, r2, mdape_val, prediction = model_func(train_X, test_X, train_y, test_y, **hyperparams)
                    test_preds = pd.Series(prediction, index=test_y.index)
                    plot_series(test_y, test_preds, model_name, fold_idx + 1)

                results['Model'].append(model_name)
                results['Fold'].append(fold_idx + 1)
                results['MAE'].append(mae)
                results['MSE'].append(mse)
                results['R2 Score'].append(r2)
                results['MdAPE'].append(mdape_val)
            except Exception as e:
                print(f"Error with {model_name} on Fold {fold_idx + 1}: {e}")

    results_df = pd.DataFrame(results)
    print("\n--- Model Performance Summary ---")
    print(results_df)
    plot_model_performance(results_df)
    plot_model_performance1(results_df)


def plot_model_performance(results_df):
    """
    Plots a comparison of model performance metrics.
    """
    metrics_to_plot = ['MAE', 'MSE', 'R2 Score', 'MdAPE']
    for metric in metrics_to_plot:
        plt.figure(figsize=(10, 6))
        pivot_df = results_df.pivot(index='Model', columns='Fold', values=metric)
        pivot_df.plot(kind='bar', title=f"Model Comparison by {metric}", figsize=(10, 6))
        plt.ylabel(metric)
        plt.xticks(rotation=45)
        plt.legend(title="Fold", loc="upper right")
        plt.show()

import matplotlib.pyplot as plt
import seaborn as sns

def plot_model_performance1(results_df):
    """
    Plots a comparison of model performance metrics.

    Improvements:
    - Better readability with distinct titles and labels.
    - Improved layout for clarity.
    - Performance trend visualization.
    """
    metrics_to_plot = ['MAE', 'MSE', 'R2 Score', 'MdAPE']
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))  # Arrange in 2x2 grid
    axes = axes.flatten()  # Flatten for easy iteration

    # Assign colors for models
    palette = sns.color_palette("tab10", len(results_df["Model"].unique()))

    for idx, metric in enumerate(metrics_to_plot):
        ax = axes[idx]
        sns.barplot(data=results_df, x='Model', y=metric, hue='Fold', ax=ax, palette=palette)

        # Formatting
        ax.set_title(f"Comparison of {metric} across Models", fontsize=14)
        ax.set_xlabel("Model", fontsize=12)
        ax.set_ylabel(metric, fontsize=12)
        ax.legend(title="Fold", loc="best")
        ax.tick_params(axis='x', rotation=30)  # Rotate x-axis labels for better visibility

    plt.tight_layout()
    plt.show()


In [62]:
# ------------------------------------------------------------------- #
#                             Entry Point
# ------------------------------------------------------------------- #
"""
A short recap on 'forecast_horizon':
    "hourly": 6,        # 1-hour (6 x 10-minute intervals)
    "short": 144,       # 1-day (144 x 10-minute intervals)
    "3-day": 432,       # 3 days (3 × 144)
    "weekly": 1008,     # 7 days (7 × 144)
    "10-day": 1440,     # 10 days (10 × 144)
    "bi-weekly": 2016,  # 14 days (14 × 144)
    "monthly": 4320,    # 30 days (30 × 144)
    "seasonal": 12960,  # 90 days (3 months × 30 days × 144)
    "yearly": 52560     # 1 year (365 × 144)
"""

if __name__ == "__main__":
    main(n_splits=3, test_size_ratio=0.1, forecast_horizon="weekly")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Course70938/earthML/finalProject/Data/IMS/allStations_10min_TEMP.csv'